import library

In [ ]:
!pip install sqlalchemy

In [ ]:
!pip install pymysql

In [64]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import requests
import re


Function to fetch HTML content of a Wikipedia page

In [65]:
def get_wikipedia_html(city):
    url = f"https://en.wikipedia.org/wiki/{city}"
    Web_page = requests.get(url)
    return Web_page.text

Population function 

In [39]:
#original 
# Function to extract population from HTML
def extract_population(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    # Find all paragraph tags
    paragraphs = soup.find_all('p')
    # Iterate over paragraphs to find population information
    for p in paragraphs:
        if 'population' in p.text.lower() or 'inhabitants' in p.text.lower():
            # Extract population value using regular expression
            population_matches = re.findall(r'\b(\d{1,3}(?:,\d{3})*(?:\.\d+)?)(?=\s*million|\s*inhabitants|\s*residents)\s*(?:million)?\s*(?:inhabitants)?\s*(?:residents)?\b', p.text)
           # population_matches = re.findall(r'\b(?:population\s+of\s+over|population\s+of|\d{1,3}(?:,\d{3})*(?:\.\d+)?)\s*(?=\s*million|\s*inhabitants|\s*residents)?(?:\s*million|\s*inhabitants|\s*residents)?\b', p.text)

            if population_matches:
                # Check if the population contains commas
                population_with_commas = population_matches[0]
                if ',' in population_with_commas:
                    # Remove commas and convert to float, then divide by million
                    population_no_commas = population_with_commas.replace(',', '')
                    return float(population_no_commas) / 1e6  # Divide by 1 million to get the value in millions
                else:
                    return float(population_with_commas)  # Return the population as float
    # If population information not found
    return float('nan')




Country function 

In [40]:
def extract_country_name(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    # Find the infobox table
    infobox = soup.find('table', class_='infobox')
    # Search for the country name within the infobox
    if infobox:
        country_cell = infobox.find('th', string='Country')
        if country_cell:
            # Get the following cell as the country name
            country_name = country_cell.find_next('td').get_text().strip()
            return country_name
    return None


Year Extraction

In [41]:
def extract_year(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    # Find all rows in the infobox
    rows = soup.select('table.infobox.ib-settlement.vcard tr')
    
    # Initialize row number
    row_number = None
    
    # Iterate over rows to find the one containing population information
    for i, row in enumerate(rows):
        if 'population' in row.text.lower():
            row_number = i + 1  # Adjust row number to 1-indexed
            break
    
    # If population row is found, extract year from the same row
    if row_number:
        year_element = soup.select_one(f'table.infobox.ib-settlement.vcard tr:nth-of-type({row_number}) th div')
        if year_element:
            year_text = year_element.get_text(strip=True)
            pattern = r'(\d{4})'
            match = re.search(pattern, year_text)
            if match:
                return match.group(1)
    
    return "Year information not found"

Coordonate Function and split into 2 float

In [42]:
def extract_coordinates(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    # Find infobox
    infobox = soup.find('table', class_='infobox')
    if infobox:
        # Find rows in infobox
        rows = infobox.find_all('tr')
        for row in rows:
            # Check if row contains 'Coordinates' or similar text
            if 'coordinates' in row.text.lower():
                # Extract coordinates from the row
                coordinates = row.find('span', class_='geo').text
                # Split coordinates into latitude and longitude
                lat, lon = map(float, coordinates.split(';'))
                return lat, lon
    # If coordinates not found
    return float('nan'), float('nan')

create table city

In [66]:

city_list = ['Munich', 'Berlin', 'Cologne', 'Hamburg', 'Frankfurt','Leipzig' , "Hannover"]




city = pd.DataFrame(city_list, columns=['city'])
#print(City)

In [67]:
city

,city
0,Munich
1,Berlin
2,Cologne
3,Hamburg
4,Frankfurt
5,Leipzig
6,Hannover


# Create the generale data frame with all collumn

In [45]:
# Function to get population, year ,coordinates, and country name for a city
def get_city_info(city):
    # Fetch HTML content of the Wikipedia page for the city
    html_doc = get_wikipedia_html(city)
    
    # Extract population for the city
    population = extract_population(html_doc)
    
    # Extract coordinates for the city
    latitude, longitude = extract_coordinates(html_doc)
    
    # Extract country name for the city
    country_name = extract_country_name(html_doc)
    
    year_population = extract_year(html_doc)
    
    return city, country_name, population,year_population, latitude, longitude

# List of cities
cities = ['Munich', 'Berlin', 'Cologne', 'Hamburg', 'Frankfurt','Leipzig' , "Hannover", "Paris","Barcelona"]

# Create a list to store city information
city_info = []

# Get information for each city and store it in the list
for city in cities:
    city_info.append(get_city_info(city))

# Create a DataFrame from the city information
city_df = pd.DataFrame(city_info, columns=['City', 'Country', 'Population (million)', "Year",'Latitude', 'Longitude'])
print(city_df)

        City  Country  Population (million)  Year  Latitude  Longitude
0     Munich  Germany              1.578132  2022  48.13750   11.57500
1     Berlin  Germany              3.850000  2024  52.52000   13.40500
2    Cologne  Germany              1.100000  2021  50.93639    6.95278
3    Hamburg  Germany              1.900000  2022  53.55000   10.00000
4  Frankfurt  Germany              0.791000  2022  50.11056    8.68222
5    Leipzig  Germany              0.628718  2021  51.34000   12.37500
6   Hannover  Germany              1.160000  2021  52.36700    9.71700
7      Paris   France              2.102650  2023  48.85667    2.35222
8  Barcelona    Spain              1.600000  2018  41.38278    2.17694


In [46]:
city_df['Year'] = pd.to_datetime(city_df['Year'], format='%Y')
city_df['Year'] = city_df['Year'].dt.strftime('%Y')

Create table population with cityname, city population , and year 

In [57]:
def get_city_info(city):
    # Fetch HTML content of the Wikipedia page for the city
    html_doc = get_wikipedia_html(city)
    
    # Extract population for the city
    population = extract_population(html_doc)
    
   
    year_population = extract_year(html_doc)
    
    return city, population,year_population

# List of cities
cities = ['Munich', 'Berlin', 'Cologne', 'Hamburg', 'Frankfurt','Leipzig' , "Hannover"]

# Create a list to store city information
city_info = []

# Get information for each city and store it in the list
for city in cities:
    city_info.append(get_city_info(city))

# Create a DataFrame from the city information
population_df = pd.DataFrame(city_info, columns=['City', 'Population (million)', "Year"])
print(population_df)

        City  Population (million)  Year
0     Munich              1.578132  2022
1     Berlin              3.850000  2024
2    Cologne              1.100000  2021
3    Hamburg              1.900000  2022
4  Frankfurt              0.791000  2022
5    Leipzig              0.628718  2021
6   Hannover              1.160000  2021


create table Geo with city country , longitude and largitude 

In [61]:
# Function to get population, year ,coordinates, and country name for a city
def get_city_info(city):
    # Fetch HTML content of the Wikipedia page for the city
    html_doc = get_wikipedia_html(city)
    
   
    
    # Extract coordinates for the city
    latitude, longitude = extract_coordinates(html_doc)
    
    # Extract country name for the city
    country_name = extract_country_name(html_doc)
  
    
    return city, country_name,  latitude, longitude

# List of cities
cities = ['Munich', 'Berlin', 'Cologne', 'Hamburg', 'Frankfurt','Leipzig' , "Hannover"]

# Create a list to store city information
city_info = []

# Get information for each city and store it in the list
for city in cities:
    city_info.append(get_city_info(city))

# Create a DataFrame from the city information

geo_df = pd.DataFrame(city_info, columns=['City', 'Country','Latitude', 'Longitude'])
print(geo_df)

        City  Country  Latitude  Longitude
0     Munich  Germany  48.13750   11.57500
1     Berlin  Germany  52.52000   13.40500
2    Cologne  Germany  50.93639    6.95278
3    Hamburg  Germany  53.55000   10.00000
4  Frankfurt  Germany  50.11056    8.68222
5    Leipzig  Germany  51.34000   12.37500
6   Hannover  Germany  52.36700    9.71700


In [ ]:
population_df.info()

In [49]:
population_df['Year'] = population_df['Year'].astype(int)

In [50]:
population_df = population_df.rename(columns={'Population (million)': 'population_million'})


In [ ]:
geo_df.info()

Connection with my sql 

In [51]:
schema = "sql_workshop"
host = "127.0.0.1"
user = "root"
password = "password"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

push the table to my sql

In [ ]:
population_df.to_sql('population_df',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [25]:
geo_df.to_sql('geo_df',
                  if_exists='append',
                  con=connection_string,
                  index=False)

7

In [70]:
city.to_sql('city',
                  if_exists='append',
                  con=connection_string,
                  index=False)

7